In [245]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.optimizers import SGD

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('ggplot')

### Step 1 & 2: Data acquisition and preparation

In [246]:
def import_data(paths):
    """ Extracts features and labels from a dataframe.

    Args: 
        path: A list of paths to a collection of .csv's in the following dataset:
            https://archive.ics.uci.edu/ml/datasets/Character+Font+Images

    Returns:
        X: A #samples x 20 x 20 numpy array containing the pixel data
           for the font samples. The pixel values are originally grayscale
           [0, 255] but scaled to the range [0, 1].

        y: A numpy array containing the ascii code for each of the sample images.
    """
    cols = ["r"+str(x)+"c"+str(y) for x in range(20) for y in range(20)]
    dfs = []
    for path in paths:
        df = pd.read_csv(path)
        dfs.append(df)
    df = pd.concat(dfs, ignore_index=True)
    features = df[cols].copy()
    scale = np.vectorize(lambda x: x / 255)
    X = scale(np.reshape(features.values, features.shape[0] * features.shape[1]))
    X = np.reshape(X, (-1, 20, 20, 1))
    y = df['m_label'].copy().as_matrix()
    return X, y


def convert_to_one_hot(y, unique_chars):
    """ Returns a nSamples x nUniqueCharacters array which is a 
        one-hot representation of y.

    Args:
        y: A 1D array containing categorical labels.
        
        unique_chars: a list of the possible values in y.

    Returns:
        one_hot_rep: a one-hot representation of y.
    """
    one_hot_rep = []
    char_to_ix = { ch:i for i,ch in enumerate(unique_chars) }
    for label in y:
        one_hot = [0 for x in range(len(unique_chars))]
        one_hot[char_to_ix[label]] = 1
        one_hot_rep.append(one_hot)
    return one_hot_rep

### Step 3: Build a keras network

In [247]:
data_paths = ["fonts/AGENCY.csv"]
X, y = import_data(data_paths)
unique_chars = list(set(y))
y = convert_to_one_hot(y, unique_chars)

model = Sequential()
# Layer 1: Convolution 2D layer with relu activations
model.add(Conv2D(32, (4, 4), activation='relu', input_shape=(20, 20, 1)))
# Layer 2: Max pooling
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
# Layer 3: Convolution
model.add(Conv2D(64, (4, 4), activation='relu'))
# Layer 4: Max pooling
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
# Layer 5: Flatten
model.add(Flatten())
# Layer 6: Dropout
model.add(Dropout(0.4))
# Layer 7: Dense with relu activation
model.add(Dense(1024, activation='relu'))
# Layer 8: Dense with softmax activation
model.add(Dense(len(unique_chars), activation='softmax'))

### Step 4: Exploration and Evaluation

Evaluate the network using cross validation (splitting data into training/testing). What is its accuracy?

In [248]:
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.3)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(np.array(x_train), np.array(y_train), epochs=50, batch_size=32)  
score = model.evaluate(np.array(x_train), np.array(y_train), batch_size=32)
print("Train accuracy:", score[1])
score = model.evaluate(np.array(x_test), np.array(y_test), batch_size=32)
print("Test accuracy:", score[1])

Epoch 1/50
702/702 [==============================] - 5s 7ms/step - loss: 5.5434 - acc: 0.0043
Epoch 2/50
702/702 [==============================] - 1s 859us/step - loss: 5.4558 - acc: 0.0185
Epoch 3/50
702/702 [==============================] - 1s 801us/step - loss: 5.0458 - acc: 0.0499
Epoch 4/50
702/702 [==============================] - 1s 990us/step - loss: 4.1814 - acc: 0.1125
Epoch 5/50
702/702 [==============================] - 0s 686us/step - loss: 3.3364 - acc: 0.2251
Epoch 6/50
702/702 [==============================] - 1s 723us/step - loss: 2.8084 - acc: 0.2849
Epoch 7/50
702/702 [==============================] - 0s 656us/step - loss: 2.4305 - acc: 0.3689
Epoch 8/50
702/702 [==============================] - 0s 693us/step - loss: 2.1501 - acc: 0.3846
Epoch 9/50
702/702 [==============================] - 0s 701us/step - loss: 1.9594 - acc: 0.4231
Epoch 10/50
702/702 [==============================] - 0s 685us/step - loss: 1.7704 - acc: 0.4701
Epoch 11/50
702/702 [==========

Create and train a different network topology (add more convolution/dropout layers, explore other types/sizes of layer). Try to find a topology that works better than the one described above.

In [ ]:
data_paths = ["fonts/AGENCY.csv"]
X, y = import_data(data_paths)
unique_chars = list(set(y))
y = convert_to_one_hot(y, unique_chars)

model = Sequential()
# Layer 1: Convolution 2D layer with relu activations
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(20, 20, 1)))
# Layer 2: Max pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
# Layer 3: Flatten
model.add(Flatten())
# Layer 4: Dropout
model.add(Dropout(0.5))
# Layer 5: Dense with relu activation
model.add(Dense(2048, activation='relu'))
# Layer 6: Dense with softmax activation
model.add(Dense(len(unique_chars), activation='softmax'))

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.3)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(np.array(x_train), np.array(y_train), epochs=50, batch_size=32)  
score = model.evaluate(np.array(x_train), np.array(y_train), batch_size=32)
print("Train accuracy:", score[1])
score = model.evaluate(np.array(x_test), np.array(y_test), batch_size=32)
print("Test accuracy:", score[1])

Epoch 1/50
702/702 [==============================] - 6s 9ms/step - loss: 5.6105 - acc: 0.0142
Epoch 2/50
702/702 [==============================] - 2s 3ms/step - loss: 4.9980 - acc: 0.0869
Epoch 3/50
702/702 [==============================] - 2s 3ms/step - loss: 3.7424 - acc: 0.2450
Epoch 4/50
702/702 [==============================] - 2s 3ms/step - loss: 2.4814 - acc: 0.4060
Epoch 5/50
702/702 [==============================] - 2s 3ms/step - loss: 1.7230 - acc: 0.5527
Epoch 6/50
702/702 [==============================] - 2s 3ms/step - loss: 1.3084 - acc: 0.6068
Epoch 7/50
702/702 [==============================] - 2s 3ms/step - loss: 1.0498 - acc: 0.6667
Epoch 8/50
702/702 [==============================] - 2s 3ms/step - loss: 0.9152 - acc: 0.7023
Epoch 9/50
702/702 [==============================] - 2s 3ms/step - loss: 0.8117 - acc: 0.7379
Epoch 10/50
702/702 [==============================] - 2s 3ms/step - loss: 0.7149 - acc: 0.7593
Epoch 11/50
702/702 [============================

Test the accuracy of your network with character inputs from a DIFFERENT font set. How does it perform?

In [ ]:
train_paths = ["fonts/AGENCY.csv"]
test_paths = ["fonts/BELL.csv"]
x_train, y_train = import_data(train_paths)
x_test, y_test = import_data(test_paths)
unique_chars = list(set(y_train).union(y_test))
y_train = convert_to_one_hot(y_train, unique_chars)
y_test = convert_to_one_hot(y_test, unique_chars)

model = Sequential()
# Layer 1: Convolution 2D layer with relu activations
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(20, 20, 1)))
# Layer 2: Max pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
# Layer 3: Flatten
model.add(Flatten())
# Layer 4: Dropout
model.add(Dropout(0.5))
# Layer 5: Dense with relu activation
model.add(Dense(2048, activation='relu'))
# Layer 6: Dense with softmax activation
model.add(Dense(len(unique_chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(np.array(x_train), np.array(y_train), epochs=50, batch_size=32)  
score = model.evaluate(np.array(x_train), np.array(y_train), batch_size=32)
print("Train accuracy:", score[1])
score = model.evaluate(np.array(x_test), np.array(y_test), batch_size=32)
print("Test accuracy:", score[1])

Epoch 1/50
1004/1004 [==============================] - 8s 8ms/step - loss: 5.5535 - acc: 0.0179A: 2s - loss: 5.5915 - ac
Epoch 2/50
1004/1004 [==============================] - 3s 3ms/step - loss: 4.3304 - acc: 0.1434
Epoch 3/50
1004/1004 [==============================] - 3s 3ms/step - loss: 2.3946 - acc: 0.3904
Epoch 4/50
1004/1004 [==============================] - 4s 4ms/step - loss: 1.5650 - acc: 0.5458
Epoch 5/50
1004/1004 [==============================] - 4s 4ms/step - loss: 1.1763 - acc: 0.6424
Epoch 6/50
1004/1004 [==============================] - 3s 3ms/step - loss: 0.9511 - acc: 0.6932
Epoch 7/50
1004/1004 [==============================] - 3s 3ms/step - loss: 0.8364 - acc: 0.7351
Epoch 8/50
 576/1004 [================>.............] - ETA: 1s - loss: 0.6686 - acc: 0.7934

Train your best network on inputs from the data from at least 2 different fonts. How does your accuracy compare to the 1-font case?

In [ ]:
train_paths = ["fonts/AGENCY.csv", "fonts/BELL.csv", "fonts/TXT.csv"]
test_paths = ["fonts/AGENCY.csv", "fonts/BELL.csv", "fonts/TXT.csv"]
X, y = import_data(train_paths)
unique_chars = list(set(y))
y = convert_to_one_hot(y, unique_chars)

model = Sequential()
# Layer 1: Convolution 2D layer with relu activations
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(20, 20, 1)))
# Layer 2: Max pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
# Layer 3: Flatten
model.add(Flatten())
# Layer 4: Dropout
model.add(Dropout(0.5))
# Layer 5: Dense with relu activation
model.add(Dense(2048, activation='relu'))
# Layer 6: Dense with softmax activation
model.add(Dense(len(unique_chars), activation='softmax'))

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.3)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(np.array(x_train), np.array(y_train), epochs=50, batch_size=32)  
score = model.evaluate(np.array(x_train), np.array(y_train), batch_size=32)
print("Train accuracy:", score[1])
score = model.evaluate(np.array(x_test), np.array(y_test), batch_size=32)
print("Test accuracy:", score[1])

What accuracy do you see when testing with inputs from a font you didn't train on now?

In [ ]:
train_paths = ["fonts/AGENCY.csv", "fonts/BELL.csv", "fonts/TXT.csv"]
test_paths = ["fonts/SNAP.csv"]
x_train, y_train = import_data(train_paths)
x_test, y_test = import_data(test_paths)
unique_chars = list(set(y_train).union(y_test))
y_train = convert_to_one_hot(y_train, unique_chars)
y_test = convert_to_one_hot(y_test, unique_chars)

model = Sequential()
# Layer 1: Convolution 2D layer with relu activations
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(20, 20, 1)))
# Layer 2: Max pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
# Layer 3: Flatten
model.add(Flatten())
# Layer 4: Dropout
model.add(Dropout(0.5))
# Layer 5: Dense with relu activation
model.add(Dense(2048, activation='relu'))
# Layer 6: Dense with softmax activation
model.add(Dense(len(unique_chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(np.array(x_train), np.array(y_train), epochs=50, batch_size=32)  
score = model.evaluate(np.array(x_train), np.array(y_train), batch_size=32)
print("Train accuracy:", score[1])
score = model.evaluate(np.array(x_test), np.array(y_test), batch_size=32)
print("Test accuracy:", score[1])